In [45]:
# function to calculate entropy
def Entropy(y):
    d = {}
    for i in y:
        if i in d:
            d[i] += 1
        else:
            d[i] = 1
    
    total = len(y)
    entropy = 0
    for ele in d:
        entropy += (-d[ele]/total)*(np.log2(d[ele]/total))
    
    return entropy

In [46]:
# function to calculate gain of a function
def gain(data,s_feature,target_feature):       
        n_entropy = 0
        vals,counts= np.unique(data[s_feature],return_counts=True)
        for i in range(len(vals)):
            k=Entropy(data.where(data[s_feature]==vals[i]).dropna()[target_feature])
            n_entropy += (counts[i]/np.sum(counts))*k
        return n_entropy

In [47]:
# function to calculate information gain
def Infogain(df,s_feature,y):
    total_entropy = Entropy(df[y])
  
    Information_Gain = total_entropy - gain(df,s_feature,y)
    
    return Information_Gain

In [48]:
#function to calculate split info
def split_info(data,f):
    elements,counts = np.unique(data[f],return_counts = True)
    split=0
    for i in range(len(elements)):
        split += (-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts))
    return split

In [49]:
# function to calculate gain ratio
def Gain_ratio(info_gain,data,best_feature):
    ratio =  info_gain/split_info(data,best_feature)
    return ratio

In [50]:
# Recursive function to implement tree in given structure
def DecisionTree(data,originaldata,features,target_feature="target",parent = None,level=0): 
    
    # Base case 1 - case where we have reached the leaf node
    if (len(np.unique(data[target_feature])) <= 1):   
        
        elements,counts = np.unique(data[target_feature],return_counts = True)
        print('Level ',level)
        if elements ==0 :
            print('Count of 0 =',np.sum(counts))
        elif elements ==1 :
            print('Count of 1 =',np.sum(counts))
        elif elements ==2 :
            print('Count of 2 =',np.sum(counts))
        print('Current Entropy is =',Entropy(data[target_feature]))
        print('Reached Leaf Node ')
        print()
        return np.unique(data[target_feature])[0]
    
    #elif len(data) == 0:                                           # checking the data is empty or not
       
    #    return np.unique(orignaldata[target_feature])
    
    # Base case 2 - case where no features are left to split upon
    elif len(features) == 0 :
        return parent
    
    else :

        P_node = np.unique(data[target_feature])                    # add all the unique values of target in parent node
        
        values = []
        for feature in features :                                   # loop over all the unused features
            v = Infogain(data,feature,target_feature)               # getting a list of information gain of all the features
            values.append(v)
        
        best_feature_index = np.argmax(values)                      # selecting best feature based on max info gain
        best_feature = features[best_feature_index]
        
        #tree = {best_feature:{}}            
        
        total_entropy = Entropy(data[target_feature])               # entropy of current node
       
        
        ratio=Gain_ratio(max(values),data,best_feature)             # gain ratio of the feature on which we split up on
        
        #Printing the tree
        
        elements,counts = np.unique(data[target_feature],return_counts = True)
        print('Level ',level)                               
        for i in range(len(elements)):
            if elements[i]==0 :
                print('count of 0  =',counts[i])
            elif elements[i]==1 :
                print('count of 1  =',counts[i])
            elif elements[i]==2 :
                print('count of 2  =',counts[i])
      
        print('Current entropy is   = ',total_entropy)  
        print('Splitting on feature ',best_feature,' with gain ratio ', ratio)
        
        print()
        
        
        new_features = features                                      # removing the function used to split      
        features=[]
        for i in new_features :                     
                                                   
            if i != best_feature :                  
                features.append(i)
        level += 1       
        new_features=None                         
        
        for vals in np.unique(data[best_feature]):   
            
            value = vals
            sub_data = (data[data[best_feature] == value]).dropna()
            
            subtree = DT(sub_data,data,features,target_feature,P_node,level)
            tree[best_feature][value] = subtree
            
        return tree

In [64]:

from sklearn import datasets                                             # importing required libraries
import pandas as pd
import numpy as np

iris = datasets.load_iris()                                              # loading iris dataset

df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']
y = pd.DataFrame(iris.target)
y.columns = ['target']


def label(val, *boundaries):                                             # labeling the data
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')

df.head()
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)    
df['target']=y                                                    # Adding the target column in dataset
print(df.columns[:-1])          

Index(['sl_labeled', 'sw_labeled', 'pl_labeled', 'pw_labeled'], dtype='object')


In [65]:
tree=DecisionTree(df, df, df.columns[:-1])     # printing steps
print()

Level  0
count of 0  = 50
count of 1  = 50
count of 2  = 50
Current entropy is   =  1.584962500721156
Splitting on feature  pw_labeled  with gain ratio  0.6996382036222091

Level  1
Count of 0 = 50
Current Entropy is = 0.0
Reached Leaf Node 

Level  1
Count of 1 = 10
Current Entropy is = 0.0
Reached Leaf Node 

Level  1
count of 1  = 40
count of 2  = 16
Current entropy is   =  0.863120568566631
Splitting on feature  pl_labeled  with gain ratio  0.4334099495621067

Level  2
Count of 1 = 1
Current Entropy is = 0.0
Reached Leaf Node 

Level  2
count of 1  = 39
count of 2  = 8
Current entropy is   =  0.6581912658132185
Splitting on feature  sl_labeled  with gain ratio  0.12674503775809332

Level  3
Count of 2 = 1
Current Entropy is = 0.0
Reached Leaf Node 

Level  3
Count of 1 = 14
Current Entropy is = 0.0
Reached Leaf Node 

Level  3
count of 1  = 23
count of 2  = 7
Current entropy is   =  0.783776947484701
Splitting on feature  sw_labeled  with gain ratio  0.07092036405148876

Level  4
C